In [68]:
# Import multiple librarys for Data request and cleanup

import requests # this library handles http requests... for eg API GET requests.
import os # enables me to retrieve api key from another file
from dotenv import load_dotenv # allows me to access env file with api key
import pandas as pd # for all things pandas
from geopy.geocoders import Nominatim # to clean up location data
import json # to parse json

load_dotenv("store.env") # loads data from env file

True

In [69]:
versionnumber = "1.0"
keywords = "planner"
locationName = "unitedkingdom" # location search criterea
employerId = ""
distanceinmiles = ""
resultsToTake = 100
resultsToSkip = 0
reed_api_key = os.getenv("reed_api_key") #retrieves api_key from .env file. This means i don't need to hard code my API key into script for improved security.

url = f"https://www.reed.co.uk/api/{versionnumber}/search?keywords={keywords}&locationName={locationName}&employerId={employerId}&distanceFromLocation={distanceinmiles}&resultsToTake={resultsToTake}&resultsToSkip={resultsToSkip}"
url

'https://www.reed.co.uk/api/1.0/search?keywords=planner&locationName=unitedkingdom&employerId=&distanceFromLocation=&resultsToTake=100&resultsToSkip=0'

In [70]:
response = requests.get(url, auth=(reed_api_key, ''))
json_for_df = [] # initialise list to extend reed_data to.
if response.status_code == 200:
    print(f"Response Success {response.status_code}") 
    reed_data = response.json() # get json and set it to reed data. Add reed data to json_for_df below
    json_for_df.extend(reed_data["results"])
    resultsToSkip = resultsToSkip + resultsToTake # increment results to skip so i don't append the same data in the loop below
    total_results = (reed_data["totalResults"])
else:
    raise Exception (f"Unsuccesfull API request: {response.status_code}") # stops execution of code and exits with error message.




Response Success 200


In [71]:
total_pages_minus_1 = ((total_results + resultsToTake - 1) // resultsToTake) - 1  # this gives the total number of pages minues 1 (because i already have the first API page) to retrieve data from.

for num_page in range(total_pages_minus_1):  # ie from 0 to the total number of pages
    print("-----")
    url = f"https://www.reed.co.uk/api/{versionnumber}/search?keywords={keywords}&locationName={locationName}&employerId={employerId}&distanceFromLocation={distanceinmiles}&resultsToTake={resultsToTake}&resultsToSkip={resultsToSkip}"
    response = requests.get(url, auth=(reed_api_key, ''))
    reed_data = response.json()
    json_for_df.extend(reed_data["results"])
    print("requesting:", url)
    resultsToSkip = resultsToSkip + resultsToTake # increment results to skip by 100 for every page

-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=planner&locationName=unitedkingdom&employerId=&distanceFromLocation=&resultsToTake=100&resultsToSkip=100
-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=planner&locationName=unitedkingdom&employerId=&distanceFromLocation=&resultsToTake=100&resultsToSkip=200
-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=planner&locationName=unitedkingdom&employerId=&distanceFromLocation=&resultsToTake=100&resultsToSkip=300
-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=planner&locationName=unitedkingdom&employerId=&distanceFromLocation=&resultsToTake=100&resultsToSkip=400
-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=planner&locationName=unitedkingdom&employerId=&distanceFromLocation=&resultsToTake=100&resultsToSkip=500
-----
requesting: https://www.reed.co.uk/api/1.0/search?keywords=planner&locationName=unitedkingdom&employerId=&distanceFromLocation=&resultsToTake=10

In [72]:
df = pd.DataFrame(json_for_df)


# Data clean-up:
df.fillna("NaN", inplace=True) # this fills empty values with NaN
df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y") # turn the date into an actual date-time date

geolocator = Nominatim(user_agent='postcode_checker')

# this function checks whether the postcode is a string & if it contains a number. If it contains a number it's likely a postcode so pass the str into the next function
def is_postcode(value):
    return isinstance(value, str) and any(character.isdigit() for character in value)


# The following function uses the postcode past into get_town & returns the 3rd from last value in the returned list. Which is often the town
def get_display_name(value):
    if is_postcode(value):
        location = geolocator.geocode(value, exactly_one=True)
        if location:
            display_name = location.raw.get("display_name", "")
            return display_name
        else:
            return value
    return value

# The following function parses the display_name from the comma and returns the -3 item in the list, if it is longer than 3 items long, as this is usually the town. If not then just returns the original value.
def get_town(value):
    value = value.split(", ")
    if len(value) >= 3:
        return value[-3]  
    else:
        return value[0]


# The following function also parses the display_name from the comma and returns the -1 item which is usually the country. If not then returns NaN
def get_country(value):
    value = value.split(", ")
    if len(value) >= 3:
        return value[-1]  
    else:
        return "NaN"


C:\Users\Henry\AppData\Local\Temp\ipykernel_2260\1238153797.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NaN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("NaN", inplace=True) # this fills empty values with NaN


In [73]:
df["displayName"] = df["locationName"].apply(get_display_name)
df


,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,displayName
0,54590610,312930,The Clancy Group,NaN,NaN,Planner,NN175UA,NaN,NaN,NaN,16/04/2025,2025-03-05,"Planner 26,000 Corby We Care That’s why we nee...",10,https://www.reed.co.uk/jobs/planner/54590610,"NN17 5UA, North Northamptonshire, England, Uni..."
1,54537437,312930,The Clancy Group,NaN,NaN,Planner,EH545FD,NaN,NaN,NaN,08/04/2025,2025-02-26,Planner Livingston We Care That’s why we need ...,14,https://www.reed.co.uk/jobs/planner/54537437,"EH54 5FD, Houstoun Industrial Estate, Knightsr..."
2,54537415,312930,The Clancy Group,NaN,NaN,Planner,NN39BX,NaN,NaN,NaN,08/04/2025,2025-02-26,Planner Competitive Salary and Package Great B...,23,https://www.reed.co.uk/jobs/planner/54537415,"NN3 9BX, Crow Lane Industrial Estate, Northamp..."
3,54623610,445637,Willmott Dixon,NaN,NaN,Senior Planner,Hitchin,NaN,NaN,NaN,22/04/2025,2025-03-11,Senior Planner Willmott Dixon are currently lo...,3,https://www.reed.co.uk/jobs/senior-planner/546...,Hitchin
4,54524565,677500,W B Power Services Limited,NaN,NaN,Project Planner,Heanor Gate,55000.0,70000.0,GBP,07/04/2025,2025-02-24,Job description Project Planner At WB Power Se...,46,https://www.reed.co.uk/jobs/project-planner/54...,Heanor Gate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,54599309,632629,Appcastenterprise,NaN,NaN,Programmatic Manager,London,NaN,NaN,NaN,17/04/2025,2025-03-06,Founded by British makeup artist and beauty en...,3,https://www.reed.co.uk/jobs/programmatic-manag...,London
1890,54626486,629474,eFinancialCareers,NaN,NaN,Manager - Quilter,London,NaN,NaN,NaN,22/04/2025,2025-03-11,About the Business Quilter plc is a leading pr...,0,https://www.reed.co.uk/jobs/manager-quilter/54...,London
1891,54591384,497203,The Guardian,NaN,NaN,Business Development Executive,E28HL,NaN,NaN,NaN,17/04/2025,2025-03-06,"Are you a driven, results-focused salesperson ...",3,https://www.reed.co.uk/jobs/business-developme...,"E2 8HL, London Borough of Hackney, Greater Lon..."
1892,54595175,629474,eFinancialCareers,NaN,NaN,Fully Remote Paraplanner,London,NaN,NaN,NaN,17/04/2025,2025-03-06,About the Business Quilter plc is a leading we...,0,https://www.reed.co.uk/jobs/fully-remote-parap...,London


In [74]:
df["townName"] = df["displayName"].apply(get_town)
df

,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,displayName,townName
0,54590610,312930,The Clancy Group,NaN,NaN,Planner,NN175UA,NaN,NaN,NaN,16/04/2025,2025-03-05,"Planner 26,000 Corby We Care That’s why we nee...",10,https://www.reed.co.uk/jobs/planner/54590610,"NN17 5UA, North Northamptonshire, England, Uni...",North Northamptonshire
1,54537437,312930,The Clancy Group,NaN,NaN,Planner,EH545FD,NaN,NaN,NaN,08/04/2025,2025-02-26,Planner Livingston We Care That’s why we need ...,14,https://www.reed.co.uk/jobs/planner/54537437,"EH54 5FD, Houstoun Industrial Estate, Knightsr...",West Lothian
2,54537415,312930,The Clancy Group,NaN,NaN,Planner,NN39BX,NaN,NaN,NaN,08/04/2025,2025-02-26,Planner Competitive Salary and Package Great B...,23,https://www.reed.co.uk/jobs/planner/54537415,"NN3 9BX, Crow Lane Industrial Estate, Northamp...",West Northamptonshire
3,54623610,445637,Willmott Dixon,NaN,NaN,Senior Planner,Hitchin,NaN,NaN,NaN,22/04/2025,2025-03-11,Senior Planner Willmott Dixon are currently lo...,3,https://www.reed.co.uk/jobs/senior-planner/546...,Hitchin,Hitchin
4,54524565,677500,W B Power Services Limited,NaN,NaN,Project Planner,Heanor Gate,55000.0,70000.0,GBP,07/04/2025,2025-02-24,Job description Project Planner At WB Power Se...,46,https://www.reed.co.uk/jobs/project-planner/54...,Heanor Gate,Heanor Gate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,54599309,632629,Appcastenterprise,NaN,NaN,Programmatic Manager,London,NaN,NaN,NaN,17/04/2025,2025-03-06,Founded by British makeup artist and beauty en...,3,https://www.reed.co.uk/jobs/programmatic-manag...,London,London
1890,54626486,629474,eFinancialCareers,NaN,NaN,Manager - Quilter,London,NaN,NaN,NaN,22/04/2025,2025-03-11,About the Business Quilter plc is a leading pr...,0,https://www.reed.co.uk/jobs/manager-quilter/54...,London,London
1891,54591384,497203,The Guardian,NaN,NaN,Business Development Executive,E28HL,NaN,NaN,NaN,17/04/2025,2025-03-06,"Are you a driven, results-focused salesperson ...",3,https://www.reed.co.uk/jobs/business-developme...,"E2 8HL, London Borough of Hackney, Greater Lon...",Greater London
1892,54595175,629474,eFinancialCareers,NaN,NaN,Fully Remote Paraplanner,London,NaN,NaN,NaN,17/04/2025,2025-03-06,About the Business Quilter plc is a leading we...,0,https://www.reed.co.uk/jobs/fully-remote-parap...,London,London


In [75]:
df["countryName"] = df["displayName"].apply(get_country)
df


,jobId,employerId,employerName,employerProfileId,employerProfileName,jobTitle,locationName,minimumSalary,maximumSalary,currency,expirationDate,date,jobDescription,applications,jobUrl,displayName,townName,countryName
0,54590610,312930,The Clancy Group,NaN,NaN,Planner,NN175UA,NaN,NaN,NaN,16/04/2025,2025-03-05,"Planner 26,000 Corby We Care That’s why we nee...",10,https://www.reed.co.uk/jobs/planner/54590610,"NN17 5UA, North Northamptonshire, England, Uni...",North Northamptonshire,United Kingdom
1,54537437,312930,The Clancy Group,NaN,NaN,Planner,EH545FD,NaN,NaN,NaN,08/04/2025,2025-02-26,Planner Livingston We Care That’s why we need ...,14,https://www.reed.co.uk/jobs/planner/54537437,"EH54 5FD, Houstoun Industrial Estate, Knightsr...",West Lothian,United Kingdom
2,54537415,312930,The Clancy Group,NaN,NaN,Planner,NN39BX,NaN,NaN,NaN,08/04/2025,2025-02-26,Planner Competitive Salary and Package Great B...,23,https://www.reed.co.uk/jobs/planner/54537415,"NN3 9BX, Crow Lane Industrial Estate, Northamp...",West Northamptonshire,United Kingdom
3,54623610,445637,Willmott Dixon,NaN,NaN,Senior Planner,Hitchin,NaN,NaN,NaN,22/04/2025,2025-03-11,Senior Planner Willmott Dixon are currently lo...,3,https://www.reed.co.uk/jobs/senior-planner/546...,Hitchin,Hitchin,NaN
4,54524565,677500,W B Power Services Limited,NaN,NaN,Project Planner,Heanor Gate,55000.0,70000.0,GBP,07/04/2025,2025-02-24,Job description Project Planner At WB Power Se...,46,https://www.reed.co.uk/jobs/project-planner/54...,Heanor Gate,Heanor Gate,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,54599309,632629,Appcastenterprise,NaN,NaN,Programmatic Manager,London,NaN,NaN,NaN,17/04/2025,2025-03-06,Founded by British makeup artist and beauty en...,3,https://www.reed.co.uk/jobs/programmatic-manag...,London,London,NaN
1890,54626486,629474,eFinancialCareers,NaN,NaN,Manager - Quilter,London,NaN,NaN,NaN,22/04/2025,2025-03-11,About the Business Quilter plc is a leading pr...,0,https://www.reed.co.uk/jobs/manager-quilter/54...,London,London,NaN
1891,54591384,497203,The Guardian,NaN,NaN,Business Development Executive,E28HL,NaN,NaN,NaN,17/04/2025,2025-03-06,"Are you a driven, results-focused salesperson ...",3,https://www.reed.co.uk/jobs/business-developme...,"E2 8HL, London Borough of Hackney, Greater Lon...",Greater London,United Kingdom
1892,54595175,629474,eFinancialCareers,NaN,NaN,Fully Remote Paraplanner,London,NaN,NaN,NaN,17/04/2025,2025-03-06,About the Business Quilter plc is a leading we...,0,https://www.reed.co.uk/jobs/fully-remote-parap...,London,London,NaN


In [76]:
df.to_excel("reed_planner_jobs.xlsx")